In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import GridSearchCV

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable

torch.manual_seed(0)

In [2]:
#Load tabular data
Dataset=pd.read_csv(r'data/TabularSummaryBatteryData.csv')

#Scale Cell life target Values
Dataset['cell_life']=np.log10(Dataset['cell_life'])

numBat1=41
numBat2=43
numBat3=40
numBat=124
test_ind = np.hstack((np.arange(0,(numBat1+numBat2),2),83))
test_edited_ind = np.delete(test_ind, 21) # Remove battery 
train_ind = np.arange(1,(numBat1+numBat2-1),2)
secondary_test_ind = np.arange(numBat-numBat3,numBat)

Training_Data=Dataset.iloc[train_ind]
Testing_Data=Dataset.iloc[test_edited_ind]
Secondary_Data=Dataset.iloc[secondary_test_ind]

#choosing model features According to table 1
target_features=['cell_life']

varmod_features=['variance_dQ_100_10']

dismod_features=['minimum_dQ_100_10',
                 'variance_dQ_100_10',
                 'skewness_dQ_100_10',
                 'kurtosis_dQ_100_10',
                 'Discharge_Cap_2',
                 'Diff_MaxCap_DC2']
    
full_features=['minimum_dQ_100_10',
               'variance_dQ_100_10',
               'Slope_Cap_Fade_2_100',
               'Intercept_Cap_Fade_2_100',
               'Diff_MaxCap_DC2',
               'Discharge_Cap_2',
               'Avg_charge_time',
               'Temp_Integ_2_100',
               'Min_IR',
               'Diff_IR_2_100']

In [3]:
#Use only the discharge model Tabular features
features = dismod_features #full_features
X_train=Training_Data[features].to_numpy()
X_test=Testing_Data[features].to_numpy()
X_sec=Secondary_Data[features].to_numpy()

#Scale
scaler = preprocessing.StandardScaler().fit(X_train)
Tabular_Train = scaler.transform(X_train)
Tabular_test = scaler.transform(X_test)
Tabular_sec = scaler.transform(X_sec)

In [4]:
#Load discharge capacity data
train_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Train.pt')
test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Test.pt')
secondary_test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Secondary_Test.pt')

print(train_Qdlin.std(),train_Qdlin.mean())
print(test_Qdlin.std(),test_Qdlin.mean())
print(secondary_test_Qdlin.std(),secondary_test_Qdlin.mean())

print(train_Qdlin.shape)
print(test_Qdlin.shape)
print(secondary_test_Qdlin.shape)

tensor(1.0000, dtype=torch.float64) tensor(-0.6982, dtype=torch.float64)
tensor(0.8786, dtype=torch.float64) tensor(-0.6188, dtype=torch.float64)
tensor(0.4862, dtype=torch.float64) tensor(-0.3183, dtype=torch.float64)
torch.Size([41, 100, 99])
torch.Size([42, 100, 99])
torch.Size([40, 100, 99])


In [5]:
y_train=Training_Data[target_features].to_numpy()
y_test=Testing_Data[target_features].to_numpy()
y_sec=Secondary_Data[target_features].to_numpy()

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
y_sec = torch.tensor(y_sec)

y_train_mean = torch.mean(y_train)
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean
y_sec = y_sec - y_train_mean
#Be sure to add y_train_mean and 10** All results

In [6]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, features, targets, tabular):
        self.features = features
        self.targets = targets
        self.tabular = tabular #Merge tabular data set
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx], self.tabular[idx]
  
    def __len__(self):
        return len(self.features)

In [7]:
class CNN(nn.Module):

    def __init__(self,inputSize):
        
        super(CNN, self).__init__()
        self.firstEpochFlag=True
        # Calculates size of first linear layer 
        # This allows the Module to be used with different number of cycles
        self.linearSize=((((inputSize-2)//2)-2)//2)*23*16+6
        
        self.convolutional_layer = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(3,3)),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2),
            #nn.BatchNorm2d(3),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        )
        self.linear_layer = nn.Sequential(
            nn.Linear(self.linearSize,120),
            #nn.Dropout(.1)
            nn.ReLU(),
            nn.Linear(120,84),  
            nn.ReLU(),
            nn.Linear(84,1),
        )

    def forward(self, x,tabular):
        #Print the shapes of the data on the first Epoch
        if(self.firstEpochFlag):print(x.shape)
        
        x = x.view(-1, 1,x.shape[1],99)
        if(self.firstEpochFlag):print(x.shape)
        
        clout = self.convolutional_layer(x)
        if(self.firstEpochFlag):print(clout.shape)
        
        flat = torch.flatten(clout, 1)
        if(self.firstEpochFlag):print(flat.shape)
            
        #Concatenate tabular features!
        flat=torch.cat((flat, tabular), -1)
        if(self.firstEpochFlag):
            print(tabular.shape)
            print(flat.shape)
            self.firstEpochFlag=False
           
        output = self.linear_layer(flat)
        return output

In [8]:
def train(model,train_loader,criterion,optimizer,epochs,device):
    for epoch in range(epochs):
        model.train()
        for inputs, targets, tabular in train_loader:
            optimizer.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device)
            tabular = tabular.to(device)
            
            outputs = model(inputs,tabular)
            outputs.to(device)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        model.eval()
        if(epoch%100 == 0):
            print('--',epoch,'-epoch--RMSE-- ',
                  np.sqrt(mean_squared_error(10**(targets.cpu().detach().numpy()+y_train_mean.numpy()), 
                                          10**(outputs.cpu().detach().numpy()+y_train_mean.numpy()))))
    return model

In [9]:
def testALL(model):
    dataloaders = [['---RMSE---Train',train_Qdlin_loader],
                   ['---RMSE---Test',test_Qdlin_loader],
                   ['---RMSE---Test Secondary',secondary_test_Qdlin_loader]]
    for dataloader in dataloaders:
        for inputs, targets,tabular in dataloader[1]:
            inputs = inputs.to(device)
            targets = targets.to(device)
            tabular = tabular.to(device)
            
            outputs = model(inputs,tabular)
            outputs.to(device)
            loss = criterion(outputs, targets)
            print(np.sqrt(mean_squared_error(10**(targets.cpu().detach().numpy()+y_train_mean.numpy()), 
                                             10**(outputs.cpu().detach().numpy()+y_train_mean.numpy()))),dataloader[0])

In [10]:
torch.set_default_dtype(torch.float64)

In [11]:
train_Dataset = CustomDataset(train_Qdlin,y_train,Tabular_Train)
test_Dataset = CustomDataset(test_Qdlin,y_test,Tabular_test)
secondary_test_Dataset = CustomDataset(secondary_test_Qdlin,y_sec,Tabular_sec)

train_Qdlin_loader = DataLoader(train_Dataset, batch_size=41, shuffle=True)
test_Qdlin_loader = DataLoader(test_Dataset, batch_size=42, shuffle=True)
secondary_test_Qdlin_loader = DataLoader(secondary_test_Dataset, batch_size=40, shuffle=True)

# Hyper-parameters

In [12]:
epochs = 5000
learning_rate =.0001
weight_decay = .001

# Train & Test

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

cuda:0


In [14]:
criterion=nn.MSELoss()
model=CNN(100) #Using only the First 100 cycles
optimizer=optim.Adam(model.parameters(),weight_decay=weight_decay, lr=learning_rate)
model = model.to(device)
model = train(model,train_Qdlin_loader,criterion,optimizer,epochs,device)

torch.Size([41, 100, 99])
torch.Size([41, 1, 100, 99])
torch.Size([41, 16, 23, 23])
torch.Size([41, 8464])
torch.Size([41, 6])
torch.Size([41, 8470])
-- 0 -epoch--RMSE--  345.414516519224
-- 100 -epoch--RMSE--  34.13337887501485
-- 200 -epoch--RMSE--  25.695042624259575
-- 300 -epoch--RMSE--  23.167757796279336
-- 400 -epoch--RMSE--  32.34280226863435
-- 500 -epoch--RMSE--  19.3650298996491
-- 600 -epoch--RMSE--  19.241104852701252
-- 700 -epoch--RMSE--  18.24891895584896
-- 800 -epoch--RMSE--  18.97845488928674
-- 900 -epoch--RMSE--  18.212282028423814
-- 1000 -epoch--RMSE--  20.371895770432502
-- 1100 -epoch--RMSE--  18.437621348892183
-- 1200 -epoch--RMSE--  21.823830696895477
-- 1300 -epoch--RMSE--  16.72272654299441
-- 1400 -epoch--RMSE--  18.48046684360875
-- 1500 -epoch--RMSE--  19.08817968773417
-- 1600 -epoch--RMSE--  15.608485506375395
-- 1700 -epoch--RMSE--  15.598586563395463
-- 1800 -epoch--RMSE--  19.402162351481934
-- 1900 -epoch--RMSE--  21.855829018810823
-- 2000 -epoc

In [15]:
testALL(model)

17.72019831441169 ---RMSE---Train
84.78603419011303 ---RMSE---Test
140.3386225637014 ---RMSE---Test Secondary


################################################################################

# Repeat using only the first 60 cycles

################################################################################

In [16]:
#Load tabular data
Dataset=pd.read_csv(r'data/TabularSummaryBatteryData-60Cycles.csv')

# Scale target Battery life By log 10
Dataset['cell_life']=np.log10(Dataset['cell_life'])

Training_Data=Dataset.iloc[train_ind]
Testing_Data=Dataset.iloc[test_edited_ind]
Secondary_Data=Dataset.iloc[secondary_test_ind]

#choosing model features According to table 1
target_features=['cell_life']

varmod_features=['variance_dQ_60_10']

dismod_features=['minimum_dQ_60_10',
                 'variance_dQ_60_10',
                 'skewness_dQ_60_10',
                 'kurtosis_dQ_60_10',
                 'Discharge_Cap_2',
                 'Diff_MaxCap_DC2']
    
full_features=['minimum_dQ_60_10',
               'variance_dQ_60_10',
               'Slope_Cap_Fade_2_60',
               'Intercept_Cap_Fade_2_60',
               'Diff_MaxCap_DC2',
               'Discharge_Cap_2',
               'Avg_charge_time',
               'Temp_Integ_2_60',
               'Min_IR',
               'Diff_IR_2_60']

In [17]:
#Use only the discharge model Tabular features
features = dismod_features #full_features
X_train=Training_Data[features].to_numpy()
X_test=Testing_Data[features].to_numpy()
X_sec=Secondary_Data[features].to_numpy()

#Scale
scaler = preprocessing.StandardScaler().fit(X_train)
Tabular_Train = scaler.transform(X_train)
Tabular_test = scaler.transform(X_test)
Tabular_sec = scaler.transform(X_sec)

In [18]:
#Load discharge capacity data
train_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Train.pt')
test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Test.pt')
secondary_test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Secondary_Test.pt')

train_Qdlin = train_Qdlin[:,:60,:]
test_Qdlin = test_Qdlin[:,:60,:]
secondary_test_Qdlin = secondary_test_Qdlin[:,:60,:]

print(train_Qdlin.std(),train_Qdlin.mean())
print(test_Qdlin.std(),test_Qdlin.mean())
print(secondary_test_Qdlin.std(),secondary_test_Qdlin.mean())

print(train_Qdlin.shape)
print(test_Qdlin.shape)
print(secondary_test_Qdlin.shape)

tensor(1.1621) tensor(-0.8226)
tensor(1.0245) tensor(-0.7291)
tensor(0.5661) tensor(-0.3582)
torch.Size([41, 60, 99])
torch.Size([42, 60, 99])
torch.Size([40, 60, 99])


In [19]:
y_train=Training_Data[target_features].to_numpy()
y_test=Testing_Data[target_features].to_numpy()
y_sec=Secondary_Data[target_features].to_numpy()

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
y_sec = torch.tensor(y_sec)

y_train_mean = torch.mean(y_train)
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean
y_sec = y_sec - y_train_mean
#Be sure to add y_train_mean and 10** All results

In [20]:
torch.set_default_dtype(torch.float64)

In [21]:
train_Dataset = CustomDataset(train_Qdlin,y_train,Tabular_Train)
test_Dataset = CustomDataset(test_Qdlin,y_test,Tabular_test)
secondary_test_Dataset = CustomDataset(secondary_test_Qdlin,y_sec,Tabular_sec)

train_Qdlin_loader = DataLoader(train_Dataset, batch_size=41, shuffle=True)
test_Qdlin_loader = DataLoader(test_Dataset, batch_size=42, shuffle=True)
secondary_test_Qdlin_loader = DataLoader(secondary_test_Dataset, batch_size=40, shuffle=True)

# Hyper-parameters

In [22]:
epochs = 5000
learning_rate =.0001
weight_decay = .001

# Train & Test

In [23]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

cuda:0


In [24]:
criterion=nn.MSELoss()
model=CNN(60) #Using only the First 60 cycles
optimizer=optim.Adam(model.parameters(),weight_decay=weight_decay, lr=learning_rate)
model = model.to(device)
model = train(model,train_Qdlin_loader,criterion,optimizer,epochs,device)

torch.Size([41, 60, 99])
torch.Size([41, 1, 60, 99])
torch.Size([41, 16, 13, 23])
torch.Size([41, 4784])
torch.Size([41, 6])
torch.Size([41, 4790])
-- 0 -epoch--RMSE--  358.7024882815354
-- 100 -epoch--RMSE--  33.84716902476151
-- 200 -epoch--RMSE--  21.016322487152966
-- 300 -epoch--RMSE--  17.570335444134496
-- 400 -epoch--RMSE--  16.321661223534
-- 500 -epoch--RMSE--  15.074609051534695
-- 600 -epoch--RMSE--  14.648473203924512
-- 700 -epoch--RMSE--  15.540556370706835
-- 800 -epoch--RMSE--  15.664397299470712
-- 900 -epoch--RMSE--  15.874757885856885
-- 1000 -epoch--RMSE--  13.951474059021557
-- 1100 -epoch--RMSE--  16.797091882787115
-- 1200 -epoch--RMSE--  12.962542382074052
-- 1300 -epoch--RMSE--  13.32044551114461
-- 1400 -epoch--RMSE--  12.476502874442502
-- 1500 -epoch--RMSE--  15.686969210534638
-- 1600 -epoch--RMSE--  12.160583605939491
-- 1700 -epoch--RMSE--  13.119884434830269
-- 1800 -epoch--RMSE--  16.007748786818826
-- 1900 -epoch--RMSE--  13.65720471782863
-- 2000 -ep

In [25]:
testALL(model)

10.892542135474853 ---RMSE---Train
186.01557094265897 ---RMSE---Test
204.6068075190482 ---RMSE---Test Secondary


################################################################################

# Repeat using only the first 40 cycles

################################################################################

In [26]:
#Load tabular data
Dataset=pd.read_csv(r'data/TabularSummaryBatteryData-40Cycles.csv')

# Scale target Battery life By log 10
Dataset['cell_life']=np.log10(Dataset['cell_life'])

Training_Data=Dataset.iloc[train_ind]
Testing_Data=Dataset.iloc[test_edited_ind]
Secondary_Data=Dataset.iloc[secondary_test_ind]

#choosing model features According to table 1
target_features=['cell_life']

varmod_features=['variance_dQ_40_10']

dismod_features=['minimum_dQ_40_10',
                 'variance_dQ_40_10',
                 'skewness_dQ_40_10',
                 'kurtosis_dQ_40_10',
                 'Discharge_Cap_2',
                 'Diff_MaxCap_DC2']
    
full_features=['minimum_dQ_40_10',
               'variance_dQ_40_10',
               'Slope_Cap_Fade_2_40',
               'Intercept_Cap_Fade_2_40',
               'Diff_MaxCap_DC2',
               'Discharge_Cap_2',
               'Avg_charge_time',
               'Temp_Integ_2_40',
               'Min_IR',
               'Diff_IR_2_40']

In [27]:
#Use only the discharge model Tabular features
features = dismod_features #full_features
X_train=Training_Data[features].to_numpy()
X_test=Testing_Data[features].to_numpy()
X_sec=Secondary_Data[features].to_numpy()

#Scale
scaler = preprocessing.StandardScaler().fit(X_train)
Tabular_Train = scaler.transform(X_train)
Tabular_test = scaler.transform(X_test)
Tabular_sec = scaler.transform(X_sec)

In [28]:
#Load discharge capacity data
train_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Train.pt')
test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Test.pt')
secondary_test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Secondary_Test.pt')

train_Qdlin = train_Qdlin[:,:40,:]
test_Qdlin = test_Qdlin[:,:40,:]
secondary_test_Qdlin = secondary_test_Qdlin[:,:40,:]

print(train_Qdlin.std(),train_Qdlin.mean())
print(test_Qdlin.std(),test_Qdlin.mean())
print(secondary_test_Qdlin.std(),secondary_test_Qdlin.mean())

print(train_Qdlin.shape)
print(test_Qdlin.shape)
print(secondary_test_Qdlin.shape)

tensor(1.1242) tensor(-0.6322)
tensor(0.9673) tensor(-0.5410)
tensor(0.5148) tensor(-0.2270)
torch.Size([41, 40, 99])
torch.Size([42, 40, 99])
torch.Size([40, 40, 99])


In [29]:
y_train=Training_Data[target_features].to_numpy()
y_test=Testing_Data[target_features].to_numpy()
y_sec=Secondary_Data[target_features].to_numpy()

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
y_sec = torch.tensor(y_sec)

y_train_mean = torch.mean(y_train)
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean
y_sec = y_sec - y_train_mean
#Be sure to add y_train_mean and 10** All results

In [30]:
torch.set_default_dtype(torch.float64)

In [31]:
train_Dataset = CustomDataset(train_Qdlin,y_train,Tabular_Train)
test_Dataset = CustomDataset(test_Qdlin,y_test,Tabular_test)
secondary_test_Dataset = CustomDataset(secondary_test_Qdlin,y_sec,Tabular_sec)

train_Qdlin_loader = DataLoader(train_Dataset, batch_size=41, shuffle=True)
test_Qdlin_loader = DataLoader(test_Dataset, batch_size=42, shuffle=True)
secondary_test_Qdlin_loader = DataLoader(secondary_test_Dataset, batch_size=40, shuffle=True)

# Hyper-parameters

In [32]:
epochs = 5000
learning_rate =.0001
weight_decay = .001

# Train & Test

In [33]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

cuda:0


In [34]:
criterion=nn.MSELoss()
model=CNN(40) #Using only the First 40 cycles
optimizer=optim.Adam(model.parameters(),weight_decay=weight_decay, lr=learning_rate)
model = model.to(device)
model = train(model,train_Qdlin_loader,criterion,optimizer,epochs,device)

torch.Size([41, 40, 99])
torch.Size([41, 1, 40, 99])
torch.Size([41, 16, 8, 23])
torch.Size([41, 2944])
torch.Size([41, 6])
torch.Size([41, 2950])
-- 0 -epoch--RMSE--  322.69592932697816
-- 100 -epoch--RMSE--  32.676021531634
-- 200 -epoch--RMSE--  21.33708917613326
-- 300 -epoch--RMSE--  18.61748503857831
-- 400 -epoch--RMSE--  16.43956947779598
-- 500 -epoch--RMSE--  15.493346628698244
-- 600 -epoch--RMSE--  15.112906719074376
-- 700 -epoch--RMSE--  14.765728519447414
-- 800 -epoch--RMSE--  14.17486305411211
-- 900 -epoch--RMSE--  16.44384473631824
-- 1000 -epoch--RMSE--  14.172456444951306
-- 1100 -epoch--RMSE--  14.368121031933258
-- 1200 -epoch--RMSE--  15.894879986102781
-- 1300 -epoch--RMSE--  13.89962063230022
-- 1400 -epoch--RMSE--  14.09343433161668
-- 1500 -epoch--RMSE--  13.995340013794094
-- 1600 -epoch--RMSE--  14.295060959402969
-- 1700 -epoch--RMSE--  13.464002779023536
-- 1800 -epoch--RMSE--  15.984058725865875
-- 1900 -epoch--RMSE--  13.527561173832757
-- 2000 -epoch-

In [35]:
testALL(model)

11.6569302628326 ---RMSE---Train
80.17404802694 ---RMSE---Test
206.35704327120192 ---RMSE---Test Secondary


################################################################################

# Repeat using only the first 20 cycles

################################################################################

In [36]:
#Load tabular data
Dataset=pd.read_csv(r'data/TabularSummaryBatteryData-20Cycles.csv')

# Scale target Battery life By log 10
Dataset['cell_life']=np.log10(Dataset['cell_life'])

Training_Data=Dataset.iloc[train_ind]
Testing_Data=Dataset.iloc[test_edited_ind]
Secondary_Data=Dataset.iloc[secondary_test_ind]

#choosing model features According to table 1
target_features=['cell_life']

varmod_features=['variance_dQ_20_10']

dismod_features=['minimum_dQ_20_10',
                 'variance_dQ_20_10',
                 'skewness_dQ_20_10',
                 'kurtosis_dQ_20_10',
                 'Discharge_Cap_2',
                 'Diff_MaxCap_DC2']
    
full_features=['minimum_dQ_20_10',
               'variance_dQ_20_10',
               'Slope_Cap_Fade_2_20',
               'Intercept_Cap_Fade_2_20',
               'Diff_MaxCap_DC2',
               'Discharge_Cap_2',
               'Avg_charge_time',
               'Temp_Integ_2_20',
               'Min_IR',
               'Diff_IR_2_20']

In [37]:
#Use only the discharge model Tabular features
features = dismod_features #full_features
X_train=Training_Data[features].to_numpy()
X_test=Testing_Data[features].to_numpy()
X_sec=Secondary_Data[features].to_numpy()

#Scale
scaler = preprocessing.StandardScaler().fit(X_train)
Tabular_Train = scaler.transform(X_train)
Tabular_test = scaler.transform(X_test)
Tabular_sec = scaler.transform(X_sec)

In [38]:
#Load discharge capacity data
train_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Train.pt')
test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Test.pt')
secondary_test_Qdlin=torch.load(r'data/DischargeCapacity-InterpolatedData_Secondary_Test.pt')

train_Qdlin = train_Qdlin[:,:20,:]
test_Qdlin = test_Qdlin[:,:20,:]
secondary_test_Qdlin = secondary_test_Qdlin[:,:20,:]

print(train_Qdlin.std(),train_Qdlin.mean())
print(test_Qdlin.std(),test_Qdlin.mean())
print(secondary_test_Qdlin.std(),secondary_test_Qdlin.mean())

print(train_Qdlin.shape)
print(test_Qdlin.shape)
print(secondary_test_Qdlin.shape)

tensor(0.0264) tensor(-0.0102)
tensor(0.0276) tensor(-0.0079)
tensor(0.0151) tensor(0.0023)
torch.Size([41, 20, 99])
torch.Size([42, 20, 99])
torch.Size([40, 20, 99])


In [39]:
y_train=Training_Data[target_features].to_numpy()
y_test=Testing_Data[target_features].to_numpy()
y_sec=Secondary_Data[target_features].to_numpy()

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
y_sec = torch.tensor(y_sec)

y_train_mean = torch.mean(y_train)
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean
y_sec = y_sec - y_train_mean
#Be sure to add y_train_mean and 10** All results

In [40]:
torch.set_default_dtype(torch.float64)

In [41]:
train_Dataset = CustomDataset(train_Qdlin,y_train,Tabular_Train)
test_Dataset = CustomDataset(test_Qdlin,y_test,Tabular_test)
secondary_test_Dataset = CustomDataset(secondary_test_Qdlin,y_sec,Tabular_sec)

train_Qdlin_loader = DataLoader(train_Dataset, batch_size=41, shuffle=True)
test_Qdlin_loader = DataLoader(test_Dataset, batch_size=42, shuffle=True)
secondary_test_Qdlin_loader = DataLoader(secondary_test_Dataset, batch_size=40, shuffle=True)

# Hyper-parameters

In [42]:
epochs = 5000
learning_rate =.0001
weight_decay = .001

# Train & Test

In [43]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

cuda:0


In [44]:
criterion=nn.MSELoss()
model=CNN(20) #Using only the First 20 cycles
optimizer=optim.Adam(model.parameters(),weight_decay=weight_decay, lr=learning_rate)
model = model.to(device)
model = train(model,train_Qdlin_loader,criterion,optimizer,epochs,device)

torch.Size([41, 20, 99])
torch.Size([41, 1, 20, 99])
torch.Size([41, 16, 3, 23])
torch.Size([41, 1104])
torch.Size([41, 6])
torch.Size([41, 1110])
-- 0 -epoch--RMSE--  345.7903297511116
-- 100 -epoch--RMSE--  299.68268093471016
-- 200 -epoch--RMSE--  279.4296688161121
-- 300 -epoch--RMSE--  243.81762816284848
-- 400 -epoch--RMSE--  188.27451597441012
-- 500 -epoch--RMSE--  135.26752072924145
-- 600 -epoch--RMSE--  100.85923598314204
-- 700 -epoch--RMSE--  80.11912053496586
-- 800 -epoch--RMSE--  67.24196814347617
-- 900 -epoch--RMSE--  62.66331151193836
-- 1000 -epoch--RMSE--  53.98655641412116
-- 1100 -epoch--RMSE--  48.65311971930468
-- 1200 -epoch--RMSE--  48.19663796322568
-- 1300 -epoch--RMSE--  45.361887845290376
-- 1400 -epoch--RMSE--  42.9542251433481
-- 1500 -epoch--RMSE--  38.714728462482356
-- 1600 -epoch--RMSE--  36.41350797466946
-- 1700 -epoch--RMSE--  37.64560403611453
-- 1800 -epoch--RMSE--  34.10960957335794
-- 1900 -epoch--RMSE--  34.027829982570836
-- 2000 -epoch--RM

In [45]:
testALL(model)

25.23467447452422 ---RMSE---Train
212.3843265967045 ---RMSE---Test
459.460953170998 ---RMSE---Test Secondary
